In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import findspark
findspark.init()

from pathlib import Path
import time

import numpy as np
import pyqtgraph as pg
import pyspark

from janelia_core.fileio import keller_lab
from janelia_core.fileio.exp_reader import read_img_file

In [3]:
image_folder = Path('/Volumes/Processing/SV4/CW_17-11-26/L1-561nm-HighFrequency_20171126_122633.corrected/Results/WeightFused/')
meta_data_folder = Path('/Volumes/Microscopy1/SV4/CW_17-11-26/L1-561nm-HighFrequency_20171126_122633')
#image_folder = Path('N:\SV4\CW_17-11-26\L1-561nm-HighFrequency_20171126_122633.corrected\Results\WeightFused')
#meta_data_folder = Path('O:\SV4\CW_17-11-26\L1-561nm-HighFrequency_20171126_122633')
#dest_dir = Path('E:\\bishop_work\\keller_vnc\\data')
new_image_folder = Path('E:\\bishop_work\\keller_vnc\\data\\images')
new_metadata_folder = Path('E:\\bishop_work\\keller_vnc\\data\\metadata')

In [4]:
#keller_lab.copy_exp(image_folder, meta_data_folder, dest_dir, update_int=500)

In [5]:
#exp = keller_lab.read_exp(image_folder, meta_data_folder)
exp = keller_lab.read_exp(new_image_folder, new_metadata_folder)

Searching for image files...
Found 3731 images.


In [6]:
img_paths = exp.ts_data['imgs']['vls']

In [7]:
def max_proj(f):
    return np.max(read_img_file(f), 0)

conf = pyspark.SparkConf().setAll([('spark.executor.memory', '5g'), ('spark.executor.cores', '60'),
                                   ('spark.cores.max', '60'), ('spark.driver.memory','200g'), 
                                  ('spark.driver.maxResultSize', '200g')])
sc = pyspark.SparkContext(conf=conf)

In [8]:
n_images = 3731
t0 = time.time()
data3 = np.array(sc.parallelize(img_paths[0:n_images]).map(max_proj).collect())
t1 = time.time()
print("Elapsed time: " + str(t1 - t0))

Elapsed time: 197.15320587158203


In [9]:
%gui qt
pg.image(data3[:,:,:])

C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_vnc\lib\site-packages\pyqtgraph-0.10.0-py3.6.egg\pyqtgraph\imageview\ImageView.py:588: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data = data[sl]


In [10]:
%gui qt
pg.image(read_img_file(img_paths[100]))

C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_vnc\lib\site-packages\pyqtgraph-0.10.0-py3.6.egg\pyqtgraph\graphicsItems\ImageItem.py:464: RuntimeWarning: invalid value encountered in double_scalars
  bins = np.arange(mn, mx+1.01*step, step, dtype=np.int)


ValueError: arange: cannot compute length